In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [4]:
dir_dict = dict()

dir_dict["data"] = "../data/"
dir_dict["raw"] = os.path.join(dir_dict["data"], "raw")
dir_dict["mid"] = os.path.join(dir_dict["data"], "mid")
dir_dict["raw_csv"] = os.path.join(dir_dict["raw"], "csv")
dir_dict["raw_json"] = os.path.join(dir_dict["raw"], "json")
dir_dict["html"] = os.path.join(dir_dict["raw"], "html")
dir_dict["completed_html"] = os.path.join(dir_dict["html"], "completed")
dir_dict["upcoming_html"] = os.path.join(dir_dict["html"], "upcoming")
dir_dict["fighterlist_html"] = os.path.join(dir_dict["html"], "fighterlist")
dir_dict["fighters_html"] = os.path.join(dir_dict["html"], "fighters")
dir_dict["completed_eventlist_html"] = os.path.join(dir_dict["completed_html"], "eventlist")
dir_dict["completed_events_html"] = os.path.join(dir_dict["completed_html"], "events")
dir_dict["completed_fights_html"] = os.path.join(dir_dict["completed_html"], "fights")
dir_dict["upcoming_eventlist_html"] = os.path.join(dir_dict["upcoming_html"], "eventlist")
dir_dict["upcoming_events_html"] = os.path.join(dir_dict["upcoming_html"], "events")
dir_dict["upcoming_fights_html"] = os.path.join(dir_dict["upcoming_html"], "fights")

In [5]:
for folderpath in dir_dict.values():
    os.makedirs(folderpath, exist_ok=True)

In [6]:
filepath = os.path.join(dir_dict["raw_json"], "completed_fights_data.json")
with open(filepath, "r") as f:
    completed_fights_lod = json.load(f)

In [7]:
len(completed_fights_lod)

6875

In [8]:
def process_fight_dict(fight_dict):

    def merge_additional_data(data_header):
        additional_dict = fight_dict.pop(data_header)
        fighter_names = additional_dict["Fighter"]
        round_order = lmap(lambda x: x.replace(" ", ""), additional_dict["Round"])
        if (fighter_names[0] == fight_dict["Fighter1 Name"]) and (fighter_names[1] == fight_dict["Fighter2 Name"]):
            fighter_order = ["Fighter1", "Fighter2"]
        elif (fighter_names[0] == fight_dict["Fighter2 Name"]) and (fighter_names[1] == fight_dict["Fighter1 Name"]):
            fighter_order = ["Fighter1", "Fighter2"]
        else:
            raise Exception(f"Fighter names not congurent in {data_header} table")

        for key, lov in list(additional_dict.items())[1:-1]:
            for i, v in enumerate(lov):
                var = f"{key}_SS" if data_header == "Significant Strikes" else key
                fight_dict[f"{fighter_order[(i % 2)]}_{round_order[i]}_{var}"] = v

    fight_dict_keys = fight_dict.keys()
    additional_data_headers = ["Totals", "Significant Strikes"]
    for data_header in additional_data_headers:
        if data_header in fight_dict_keys:
            merge_additional_data(data_header)

    return fight_dict

In [9]:
res_fight_dict_list = lmap(process_fight_dict, completed_fights_lod)

In [10]:
len(res_fight_dict_list)

6875

In [11]:
fights_df = pd.DataFrame(res_fight_dict_list)

In [12]:
fights_df.memory_usage(deep=True).sum() / 10**6

77.408445

In [13]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_fights_data.csv")
fights_df.to_csv(filepath, index=False)

In [14]:
fights_df = pd.read_csv(filepath)

/tmp/ipykernel_254618/339421766.py:1: DtypeWarning: Columns (223) have mixed types. Specify dtype option on import or set low_memory=False.
  fights_df = pd.read_csv(filepath)


In [15]:
fights_df["Event ID"] = fights_df["Event Url"].map(lambda s: os.path.split(s)[1])

In [16]:
fights_df["Fighter1 ID"] = fights_df["Fighter1 Url"].map(lambda s: os.path.split(s)[1])
fights_df["Fighter2 ID"] = fights_df["Fighter2 Url"].map(lambda s: os.path.split(s)[1])

In [17]:
fights_df = fights_df.drop(columns=["Event Url", "Fighter1 Url", "Fighter2 Url"])

In [18]:
fights_df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,Time format,Referee,Details,Fighter1_Overall_KD,Fighter2_Overall_KD,Fighter1_Round1_KD,Fighter2_Round1_KD,Fighter1_Round2_KD,Fighter2_Round2_KD,Fighter1_Overall_Sig. str.,Fighter2_Overall_Sig. str.,Fighter1_Round1_Sig. str.,Fighter2_Round1_Sig. str.,Fighter1_Round2_Sig. str.,Fighter2_Round2_Sig. str.,Fighter1_Overall_Sig. str. %,Fighter2_Overall_Sig. str. %,Fighter1_Round1_Sig. str. %,Fighter2_Round1_Sig. str. %,Fighter1_Round2_Sig. str. %,Fighter2_Round2_Sig. str. %,Fighter1_Overall_Total str.,Fighter2_Overall_Total str.,Fighter1_Round1_Total str.,Fighter2_Round1_Total str.,Fighter1_Round2_Total str.,Fighter2_Round2_Total str.,Fighter1_Overall_Td,Fighter2_Overall_Td,Fighter1_Round1_Td,Fighter2_Round1_Td,Fighter1_Round2_Td,Fighter2_Round2_Td,Fighter1_Overall_Td %,Fighter2_Overall_Td %,Fighter1_Round1_Td %,Fighter2_Round1_Td %,Fighter1_Round2_Td %,Fighter2_Round2_Td %,Fighter1_Overall_Sub. att,Fighter2_Overall_Sub. att,Fighter1_Round1_Sub. att,Fighter2_Round1_Sub. att,Fighter1_Round2_Sub. att,Fighter2_Round2_Sub. att,Fighter1_Overall_Rev.,Fighter2_Overall_Rev.,Fighter1_Round1_Rev.,Fighter2_Round1_Rev.,Fighter1_Round2_Rev.,Fighter2_Round2_Rev.,Fighter1_Overall_Ctrl,Fighter2_Overall_Ctrl,Fighter1_Round1_Ctrl,Fighter2_Round1_Ctrl,Fighter1_Round2_Ctrl,Fighter2_Round2_Ctrl,Fighter1_Overall_Sig. str_SS,Fighter2_Overall_Sig. str_SS,Fighter1_Round1_Sig. str_SS,Fighter2_Round1_Sig. str_SS,Fighter1_Round2_Sig. str_SS,Fighter2_Round2_Sig. str_SS,Fighter1_Overall_Sig. str. %_SS,Fighter2_Overall_Sig. str. %_SS,Fighter1_Round1_Sig. str. %_SS,Fighter2_Round1_Sig. str. %_SS,Fighter1_Round2_Sig. str. %_SS,Fighter2_Round2_Sig. str. %_SS,Fighter1_Overall_Head_SS,Fighter2_Overall_Head_SS,Fighter1_Round1_Head_SS,Fighter2_Round1_Head_SS,Fighter1_Round2_Head_SS,Fighter2_Round2_Head_SS,Fighter1_Overall_Body_SS,Fighter2_Overall_Body_SS,Fighter1_Round1_Body_SS,Fighter2_Round1_Body_SS,Fighter1_Round2_Body_SS,Fighter2_Round2_Body_SS,Fighter1_Overall_Leg_SS,Fighter2_Overall_Leg_SS,Fighter1_Round1_Leg_SS,Fighter2_Round1_Leg_SS,Fighter1_Round2_Leg_SS,Fighter2_Round2_Leg_SS,Fighter1_Overall_Distance_SS,Fighter2_Overall_Distance_SS,Fighter1_Round1_Distance_SS,Fighter2_Round1_Distance_SS,Fighter1_Round2_Distance_SS,Fighter2_Round2_Distance_SS,Fighter1_Overall_Clinch_SS,Fighter2_Overall_Clinch_SS,Fighter1_Round1_Clinch_SS,Fighter2_Round1_Clinch_SS,Fighter1_Round2_Clinch_SS,Fighter2_Round2_Clinch_SS,Fighter1_Overall_Ground_SS,Fighter2_Overall_Ground_SS,Fighter1_Round1_Ground_SS,Fighter2_Round1_Ground_SS,Fighter1_Round2_Ground_SS,Fighter2_Round2_Ground_SS,Fighter1_Round3_KD,Fighter2_Round3_KD,Fighter1_Round3_Sig. str.,Fighter2_Round3_Sig. str.,Fighter1_Round3_Sig. str. %,Fighter2_Round3_Sig. str. %,Fighter1_Round3_Total str.,Fighter2_Round3_Total str.,Fighter1_Round3_Td,Fighter2_Round3_Td,Fighter1_Round3_Td %,Fighter2_Round3_Td %,Fighter1_Round3_Sub. att,Fighter2_Round3_Sub. att,Fighter1_Round3_Rev.,Fighter2_Round3_Rev.,Fighter1_Round3_Ctrl,Fighter2_Round3_Ctrl,Fighter1_Round3_Sig. str_SS,Fighter2_Round3_Sig. str_SS,Fighter1_Round3_Sig. str. %_SS,Fighter2_Round3_Sig. str. %_SS,Fighter1_Round3_Head_SS,Fighter2_Round3_Head_SS,Fighter1_Round3_Body_SS,Fighter2_Round3_Body_SS,Fighter1_Round3_Leg_SS,Fighter2_Round3_Leg_SS,Fighter1_Round3_Distance_SS,Fighter2_Round3_Distance_SS,Fighter1_Round3_Clinch_SS,Fighter2_Round3_Clinch_SS,Fighter1_Round3_Ground_SS,Fighter2_Round3_Ground_SS,Fighter1_Round4_KD,Fighter2_Round4_KD,Fighter1_Round4_Sig. str.,Fighter2_Round4_Sig. str.,Fighter1_Round4_Sig. str. %,Fighter2_Round4_Sig. str. %,Fighter1_Round4_Total str.,Fighter2_Round4_Total str.,Fighter1_Round4_Td,Fighter2_Round4_Td,Fighter1_Round4_Td %,Fighter2_Round4_Td %,Fighter1_Round4_Sub. att,Fighter2_Round4_Sub. att,Fighter1_Round4_Rev.,Fighter2_Round4_Rev.,Fighter1_Round4_Ctrl,Fighter2_Round4_Ctrl,Fighter1_Round4_Sig. str_SS,Fighter2_Round4_Sig. str_SS,Fighter1_Round4_Sig. str. %_

### Join Weight Class data

In [19]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_fight_urls.csv")
fights_df2 = pd.read_csv(filepath)

In [20]:
fights_df2["Fight ID"] = fights_df2["Fight Url"].map(lambda s: os.path.split(s)[1])
fights_df2 = fights_df2.drop(columns=["Fight Url"])

In [21]:
fights_df2.head()

,Weight Class,Fight ID
0,Lightweight,81f287d035190506
1,Heavyweight,1f0254eeb2025e4f
2,Lightweight,4942323a5d5a6594
3,Women's Strawweight,95ead23dc0c73285
4,Featherweight,164f56eb25d6a9b4


In [22]:
fights_df = fights_df.merge(fights_df2, on="Fight ID", how="left")

In [23]:
fights_df["Weight Class"].isnull().sum()

0

### Join Events data

In [24]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_events.csv")
events_df = pd.read_csv(filepath)

In [25]:
events_df.head()

,name,date,location,url
0,UFC Fight Night: Cannonier vs. Strickland,"December 17, 2022","Las Vegas, Nevada, USA",http://ufcstats.com/event-details/56ec58954158...
1,UFC 282: Blachowicz vs. Ankalaev,"December 10, 2022","Las Vegas, Nevada, USA",http://ufcstats.com/event-details/f65a0eb902f9...
2,UFC Fight Night: Thompson vs. Holland,"December 03, 2022","Orlando, Florida, USA",http://ufcstats.com/event-details/b23388ff8ac6...
3,UFC Fight Night: Nzechukwu vs. Cutelaba,"November 19, 2022","Las Vegas, Nevada, USA",http://ufcstats.com/event-details/012fc7cd0779...
4,UFC 281: Adesanya vs. Pereira,"November 12, 2022","New York City, New York, USA",http://ufcstats.com/event-details/b3b6e80b7d5f...


In [26]:
events_df["Event ID"] = events_df["url"].map(lambda s: os.path.split(s)[1])
events_df = events_df.drop(columns=["name", "url"])

In [27]:
events_df.head()

,date,location,Event ID
0,"December 17, 2022","Las Vegas, Nevada, USA",56ec58954158966a
1,"December 10, 2022","Las Vegas, Nevada, USA",f65a0eb902f9476b
2,"December 03, 2022","Orlando, Florida, USA",b23388ff8ac6637b
3,"November 19, 2022","Las Vegas, Nevada, USA",012fc7cd0779c09a
4,"November 12, 2022","New York City, New York, USA",b3b6e80b7d5f8f0d


In [28]:
fights_df = fights_df.merge(events_df, on="Event ID", how="left")

In [29]:
fights_df.iloc[:,-2:].isnull().sum()

date        0
location    0
dtype: int64

In [30]:
fights_df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,Time format,Referee,Details,Fighter1_Overall_KD,Fighter2_Overall_KD,Fighter1_Round1_KD,Fighter2_Round1_KD,Fighter1_Round2_KD,Fighter2_Round2_KD,Fighter1_Overall_Sig. str.,Fighter2_Overall_Sig. str.,Fighter1_Round1_Sig. str.,Fighter2_Round1_Sig. str.,Fighter1_Round2_Sig. str.,Fighter2_Round2_Sig. str.,Fighter1_Overall_Sig. str. %,Fighter2_Overall_Sig. str. %,Fighter1_Round1_Sig. str. %,Fighter2_Round1_Sig. str. %,Fighter1_Round2_Sig. str. %,Fighter2_Round2_Sig. str. %,Fighter1_Overall_Total str.,Fighter2_Overall_Total str.,Fighter1_Round1_Total str.,Fighter2_Round1_Total str.,Fighter1_Round2_Total str.,Fighter2_Round2_Total str.,Fighter1_Overall_Td,Fighter2_Overall_Td,Fighter1_Round1_Td,Fighter2_Round1_Td,Fighter1_Round2_Td,Fighter2_Round2_Td,Fighter1_Overall_Td %,Fighter2_Overall_Td %,Fighter1_Round1_Td %,Fighter2_Round1_Td %,Fighter1_Round2_Td %,Fighter2_Round2_Td %,Fighter1_Overall_Sub. att,Fighter2_Overall_Sub. att,Fighter1_Round1_Sub. att,Fighter2_Round1_Sub. att,Fighter1_Round2_Sub. att,Fighter2_Round2_Sub. att,Fighter1_Overall_Rev.,Fighter2_Overall_Rev.,Fighter1_Round1_Rev.,Fighter2_Round1_Rev.,Fighter1_Round2_Rev.,Fighter2_Round2_Rev.,Fighter1_Overall_Ctrl,Fighter2_Overall_Ctrl,Fighter1_Round1_Ctrl,Fighter2_Round1_Ctrl,Fighter1_Round2_Ctrl,Fighter2_Round2_Ctrl,Fighter1_Overall_Sig. str_SS,Fighter2_Overall_Sig. str_SS,Fighter1_Round1_Sig. str_SS,Fighter2_Round1_Sig. str_SS,Fighter1_Round2_Sig. str_SS,Fighter2_Round2_Sig. str_SS,Fighter1_Overall_Sig. str. %_SS,Fighter2_Overall_Sig. str. %_SS,Fighter1_Round1_Sig. str. %_SS,Fighter2_Round1_Sig. str. %_SS,Fighter1_Round2_Sig. str. %_SS,Fighter2_Round2_Sig. str. %_SS,Fighter1_Overall_Head_SS,Fighter2_Overall_Head_SS,Fighter1_Round1_Head_SS,Fighter2_Round1_Head_SS,Fighter1_Round2_Head_SS,Fighter2_Round2_Head_SS,Fighter1_Overall_Body_SS,Fighter2_Overall_Body_SS,Fighter1_Round1_Body_SS,Fighter2_Round1_Body_SS,Fighter1_Round2_Body_SS,Fighter2_Round2_Body_SS,Fighter1_Overall_Leg_SS,Fighter2_Overall_Leg_SS,Fighter1_Round1_Leg_SS,Fighter2_Round1_Leg_SS,Fighter1_Round2_Leg_SS,Fighter2_Round2_Leg_SS,Fighter1_Overall_Distance_SS,Fighter2_Overall_Distance_SS,Fighter1_Round1_Distance_SS,Fighter2_Round1_Distance_SS,Fighter1_Round2_Distance_SS,Fighter2_Round2_Distance_SS,Fighter1_Overall_Clinch_SS,Fighter2_Overall_Clinch_SS,Fighter1_Round1_Clinch_SS,Fighter2_Round1_Clinch_SS,Fighter1_Round2_Clinch_SS,Fighter2_Round2_Clinch_SS,Fighter1_Overall_Ground_SS,Fighter2_Overall_Ground_SS,Fighter1_Round1_Ground_SS,Fighter2_Round1_Ground_SS,Fighter1_Round2_Ground_SS,Fighter2_Round2_Ground_SS,Fighter1_Round3_KD,Fighter2_Round3_KD,Fighter1_Round3_Sig. str.,Fighter2_Round3_Sig. str.,Fighter1_Round3_Sig. str. %,Fighter2_Round3_Sig. str. %,Fighter1_Round3_Total str.,Fighter2_Round3_Total str.,Fighter1_Round3_Td,Fighter2_Round3_Td,Fighter1_Round3_Td %,Fighter2_Round3_Td %,Fighter1_Round3_Sub. att,Fighter2_Round3_Sub. att,Fighter1_Round3_Rev.,Fighter2_Round3_Rev.,Fighter1_Round3_Ctrl,Fighter2_Round3_Ctrl,Fighter1_Round3_Sig. str_SS,Fighter2_Round3_Sig. str_SS,Fighter1_Round3_Sig. str. %_SS,Fighter2_Round3_Sig. str. %_SS,Fighter1_Round3_Head_SS,Fighter2_Round3_Head_SS,Fighter1_Round3_Body_SS,Fighter2_Round3_Body_SS,Fighter1_Round3_Leg_SS,Fighter2_Round3_Leg_SS,Fighter1_Round3_Distance_SS,Fighter2_Round3_Distance_SS,Fighter1_Round3_Clinch_SS,Fighter2_Round3_Clinch_SS,Fighter1_Round3_Ground_SS,Fighter2_Round3_Ground_SS,Fighter1_Round4_KD,Fighter2_Round4_KD,Fighter1_Round4_Sig. str.,Fighter2_Round4_Sig. str.,Fighter1_Round4_Sig. str. %,Fighter2_Round4_Sig. str. %,Fighter1_Round4_Total str.,Fighter2_Round4_Total str.,Fighter1_Round4_Td,Fighter2_Round4_Td,Fighter1_Round4_Td %,Fighter2_Round4_Td %,Fighter1_Round4_Sub. att,Fighter2_Round4_Sub. att,Fighter1_Round4_Rev.,Fighter2_Round4_Rev.,Fighter1_Round4_Ctrl,Fighter2_Round4_Ctrl,Fighter1_Round4_Sig. str_SS,Fighter2_Round4_Sig. str_SS,Fighter1_Round4_Sig. str. %_

### Join Fighters data

In [31]:
for i in [1, 2]:
    filepath = os.path.join(dir_dict["raw_csv"], "fighters_data.csv")
    fighters_df = pd.read_csv(filepath)
    fighters_df = fighters_df.drop(columns=["Name"])
    print(fighters_df.shape, fighters_df["Record"].isnull().sum())
    fighters_df.columns = [f"Fighter{i} {col}" if "Fighter" not in col else col.replace("Fighter", f"Fighter{i}") 
                            for col in fighters_df.columns]
    
    fights_df = fights_df.merge(fighters_df, on=f"Fighter{i} ID", how="left")

(2389, 16) 0
(2389, 16) 0


In [32]:
fights_df[["Fighter1 Record", "Fighter2 Record"]].isnull().sum()

Fighter1 Record    0
Fighter2 Record    0
dtype: int64

In [33]:
fights_df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,Time format,Referee,Details,Fighter1_Overall_KD,Fighter2_Overall_KD,Fighter1_Round1_KD,Fighter2_Round1_KD,Fighter1_Round2_KD,Fighter2_Round2_KD,Fighter1_Overall_Sig. str.,Fighter2_Overall_Sig. str.,Fighter1_Round1_Sig. str.,Fighter2_Round1_Sig. str.,Fighter1_Round2_Sig. str.,Fighter2_Round2_Sig. str.,Fighter1_Overall_Sig. str. %,Fighter2_Overall_Sig. str. %,Fighter1_Round1_Sig. str. %,Fighter2_Round1_Sig. str. %,Fighter1_Round2_Sig. str. %,Fighter2_Round2_Sig. str. %,Fighter1_Overall_Total str.,Fighter2_Overall_Total str.,Fighter1_Round1_Total str.,Fighter2_Round1_Total str.,Fighter1_Round2_Total str.,Fighter2_Round2_Total str.,Fighter1_Overall_Td,Fighter2_Overall_Td,Fighter1_Round1_Td,Fighter2_Round1_Td,Fighter1_Round2_Td,Fighter2_Round2_Td,Fighter1_Overall_Td %,Fighter2_Overall_Td %,Fighter1_Round1_Td %,Fighter2_Round1_Td %,Fighter1_Round2_Td %,Fighter2_Round2_Td %,Fighter1_Overall_Sub. att,Fighter2_Overall_Sub. att,Fighter1_Round1_Sub. att,Fighter2_Round1_Sub. att,Fighter1_Round2_Sub. att,Fighter2_Round2_Sub. att,Fighter1_Overall_Rev.,Fighter2_Overall_Rev.,Fighter1_Round1_Rev.,Fighter2_Round1_Rev.,Fighter1_Round2_Rev.,Fighter2_Round2_Rev.,Fighter1_Overall_Ctrl,Fighter2_Overall_Ctrl,Fighter1_Round1_Ctrl,Fighter2_Round1_Ctrl,Fighter1_Round2_Ctrl,Fighter2_Round2_Ctrl,Fighter1_Overall_Sig. str_SS,Fighter2_Overall_Sig. str_SS,Fighter1_Round1_Sig. str_SS,Fighter2_Round1_Sig. str_SS,Fighter1_Round2_Sig. str_SS,Fighter2_Round2_Sig. str_SS,Fighter1_Overall_Sig. str. %_SS,Fighter2_Overall_Sig. str. %_SS,Fighter1_Round1_Sig. str. %_SS,Fighter2_Round1_Sig. str. %_SS,Fighter1_Round2_Sig. str. %_SS,Fighter2_Round2_Sig. str. %_SS,Fighter1_Overall_Head_SS,Fighter2_Overall_Head_SS,Fighter1_Round1_Head_SS,Fighter2_Round1_Head_SS,Fighter1_Round2_Head_SS,Fighter2_Round2_Head_SS,Fighter1_Overall_Body_SS,Fighter2_Overall_Body_SS,Fighter1_Round1_Body_SS,Fighter2_Round1_Body_SS,Fighter1_Round2_Body_SS,Fighter2_Round2_Body_SS,Fighter1_Overall_Leg_SS,Fighter2_Overall_Leg_SS,Fighter1_Round1_Leg_SS,Fighter2_Round1_Leg_SS,Fighter1_Round2_Leg_SS,Fighter2_Round2_Leg_SS,Fighter1_Overall_Distance_SS,Fighter2_Overall_Distance_SS,Fighter1_Round1_Distance_SS,Fighter2_Round1_Distance_SS,Fighter1_Round2_Distance_SS,Fighter2_Round2_Distance_SS,Fighter1_Overall_Clinch_SS,Fighter2_Overall_Clinch_SS,Fighter1_Round1_Clinch_SS,Fighter2_Round1_Clinch_SS,Fighter1_Round2_Clinch_SS,Fighter2_Round2_Clinch_SS,Fighter1_Overall_Ground_SS,Fighter2_Overall_Ground_SS,Fighter1_Round1_Ground_SS,Fighter2_Round1_Ground_SS,Fighter1_Round2_Ground_SS,Fighter2_Round2_Ground_SS,Fighter1_Round3_KD,Fighter2_Round3_KD,Fighter1_Round3_Sig. str.,Fighter2_Round3_Sig. str.,Fighter1_Round3_Sig. str. %,Fighter2_Round3_Sig. str. %,Fighter1_Round3_Total str.,Fighter2_Round3_Total str.,Fighter1_Round3_Td,Fighter2_Round3_Td,Fighter1_Round3_Td %,Fighter2_Round3_Td %,Fighter1_Round3_Sub. att,Fighter2_Round3_Sub. att,Fighter1_Round3_Rev.,Fighter2_Round3_Rev.,Fighter1_Round3_Ctrl,Fighter2_Round3_Ctrl,Fighter1_Round3_Sig. str_SS,Fighter2_Round3_Sig. str_SS,Fighter1_Round3_Sig. str. %_SS,Fighter2_Round3_Sig. str. %_SS,Fighter1_Round3_Head_SS,Fighter2_Round3_Head_SS,Fighter1_Round3_Body_SS,Fighter2_Round3_Body_SS,Fighter1_Round3_Leg_SS,Fighter2_Round3_Leg_SS,Fighter1_Round3_Distance_SS,Fighter2_Round3_Distance_SS,Fighter1_Round3_Clinch_SS,Fighter2_Round3_Clinch_SS,Fighter1_Round3_Ground_SS,Fighter2_Round3_Ground_SS,Fighter1_Round4_KD,Fighter2_Round4_KD,Fighter1_Round4_Sig. str.,Fighter2_Round4_Sig. str.,Fighter1_Round4_Sig. str. %,Fighter2_Round4_Sig. str. %,Fighter1_Round4_Total str.,Fighter2_Round4_Total str.,Fighter1_Round4_Td,Fighter2_Round4_Td,Fighter1_Round4_Td %,Fighter2_Round4_Td %,Fighter1_Round4_Sub. att,Fighter2_Round4_Sub. att,Fighter1_Round4_Rev.,Fighter2_Round4_Rev.,Fighter1_Round4_Ctrl,Fighter2_Round4_Ctrl,Fighter1_Round4_Sig. str_SS,Fighter2_Round4_Sig. str_SS,Fighter1_Round4_Sig. str. %_

In [34]:
fights_df.isnull().sum()

Event Name            0
Fight ID              0
Fighter1 Status       0
Fighter1 Name         0
Fighter2 Status       0
                     ..
Fighter2 Str. Def     0
Fighter2 TD Avg.      0
Fighter2 TD Acc.      0
Fighter2 TD Def.      0
Fighter2 Sub. Avg.    0
Length: 257, dtype: int64

In [35]:
filepath = os.path.join(dir_dict["mid"], "completed_fights.csv")
fights_df.to_csv(filepath, index=False)